In [ ]:
%pip install duckdb numpy pandas

In [17]:
file_path = "/workspaces/Delivery-Time-Crisis-Analysis/Datasets"

In [1]:
import duckdb

conn = duckdb.connect("main.db")

In [20]:
conn.execute(f"""CREATE OR REPLACE TABLE events 
    AS 
    SELECT * FROM read_csv("{file_path}/ds_blinkit_events.csv")
    """)

conn.execute("SELECT * FROM events LIMIT 3").fetch_df()

,event_id,zone_id,event_type,start_time,end_time
0,EVT00001,106,Rain,2024-01-04 17:00:00,2024-01-04 19:00:00
1,EVT00002,101,Rain,2024-01-04 17:00:00,2024-01-04 19:00:00
2,EVT00003,106,Traffic_Jam,2024-01-09 14:00:00,2024-01-09 15:30:00


In [21]:
conn.execute(f"""CREATE OR REPLACE TABLE orders 
    AS 
    SELECT * FROM read_csv("{file_path}/ds_blinkit_orders.csv")
    """)

conn.execute("SELECT * FROM orders LIMIT 3").fetch_df()

,order_id,zone_id,store_id,order_time,delivery_time_mins,status,cancellation_reason
0,ORD1000000,101,1,2024-01-01 00:47:00,9.40,Delivered,None
1,ORD1000001,101,1,2024-01-01 00:08:00,8.89,Delivered,None
2,ORD1000002,101,3,2024-01-01 00:47:00,9.52,Delivered,None


In [22]:
conn.execute(f"""CREATE OR REPLACE TABLE store_load 
    AS 
    SELECT * FROM read_csv("{file_path}/ds_blinkit_store_load.csv")
    """)

conn.execute("SELECT * FROM store_load LIMIT 3").fetch_df()

,store_id,timestamp,active_riders,orders_in_queue,avg_rider_wait_time
0,1,2024-01-01,11,0,1.01
1,2,2024-01-01,6,3,1.50
2,3,2024-01-01,6,3,0.74


In [24]:
conn.execute("""
SELECT
DATE(order_time) AS date,
AVG(delivery_time_mins)
FROM orders
WHERE status = 'Delivered'
GROUP BY DATE(order_time)
""").fetch_df()

,date,avg(delivery_time_mins)
0,2024-01-01,9.146224
1,2024-01-02,9.101640
2,2024-01-03,10.127479
3,2024-01-04,9.142636
4,2024-01-05,10.114771
5,2024-01-06,9.147460
6,2024-01-07,9.112966
7,2024-01-08,9.127819
8,2024-01-09,9.135982
9,2024-01-10,9.111990
